In [1]:
import os
import openai
import time
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
from ute import init_openai, read_page_list, rdweb, embedpagelist, sepstr
from ute import SEP, get_embedding, cosine_similarity
import numpy as np


(Client, LLM)=init_openai(model="gpt-4-1106-preview")
webfolder="general"
datafolder="data"+"/"+webfolder+"/"
DataFile=datafolder+"data"+".txt"
TextListFile=datafolder+webfolder+"_textlist.txt"
UploadListFile=datafolder+webfolder+"_uploadlist.txt"
UploadIDFile=datafolder+webfolder+"_uploadid.txt"
EmbeddingFile=datafolder+webfolder+"_embedding.npz"
#
TextSimilarity=None
topN=None
N=3
#
Thread=None
Run=None
Messages=[]
RunIDs=[]


In [2]:
# Time stamp to local time
def ts2lt(ts):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ts))


In [3]:
%run globals.py



In [4]:
s=showglobals()
mdc(s,12)

<span style='font-size: 12px;'>

|Contents|File Name|
|--|--|
|Web folder name|general|
|Data folder name|data/general/|
|Data file name|data/general/data.txt|
|Text list file name|data/general/general_textlist.txt|
|Upload list file name|data/general/general_uploadlist.txt|
|Upload ID file name|data/general/general_uploadid.txt|
|Embedding file name|data/general/general_embedding.npz|
|Client|<openai.OpenAI object at 0x108b04810>|
|LLM|text-embedding-ada-002|




|Variable name|Description|
|--|--|
|`webfolder`|Web folder name|
|`datafolder`|Data folder name|
|`DataFile`|Data file name|
|`TextListFile`|Text list file name|
|`UploadListFile`|Upload list file name|
|`UploadIDFile`|Upload ID file name|
|`EmbeddingFile`|Embedding file name|
|`Client`|OpenAI client|
|`LLM`|Language model|
`UploadList`|List of file names to upload|
`UploadID`|List of OpenAI file IDs (created when files are uploaded)|
`TextList`|List of text files (as pairs of two lines, URL and title)|
`Embedding`|Embedding matrix|
`TextSimilarity`|Similarity vector|
`topN`|`TextSimilarity` indices of the N best-matching pages|
`N`|Number of best-matching pages to display|
`Thread`|Current thread object|
`Run`|Current run object|
`Messages`|List of messages on the current thread|
`RunIDs`|List of run IDs on the current thread|


</span>

## Save the text files locally and upload to OpenAI ##
We upload the files to the file repository of the Client. They are not yet associated with any Thread nor Assistant.  We will initiate the Assistant next and a new thread is created after the Query is set.

In [5]:
# The following function reads the names of the URLs already uploaded to OpenAI
def read_upload_list(UploadListFile):
    UploadList=[]
    if os.path.exists(UploadListFile):
        with open(UploadListFile, 'r') as f:
            UploadList = f.read().splitlines()
    return UploadList

UploadList=read_upload_list(UploadListFile)

# The following function reads the IDs of the URLs already uploaded to OpenAI
def read_upload_id(UploadIDFile):
    UploadID=[]
    if os.path.exists(UploadIDFile):
        with open(UploadIDFile, 'r') as f:
            UploadID = f.read().splitlines()
    return UploadID

UploadID=read_upload_id(UploadIDFile)
#
# The following function extracts a file name from the URL.  The file name is
# used to name the text file that will be uploaded to OpenAI
# The file name is also used to check if the file has already been uploaded
# Use the last part of the URL as the file name
def get_file_name(url):
    url_parts=url.split("/")
    filename=url_parts[-1]
    return filename

# Read the URL addresses to be read into text files from the file TextListFile
# Ignore the URLs that have already been uploaded to OpenAI

TextList=read_page_list(TextListFile)
NewUploads=[]
NewUploadIDs=[]
for (itext,pair) in enumerate(TextList):
    (url,title)=pair
    txtfile=get_file_name(url)+".txt"
    if txtfile in UploadList:
        print("Skipping %s because it has already been uploaded"%txtfile)
        continue
    # Read the text from the URL address and save into the text file
    rdweb(url, datafolder+txtfile)
    print("Uploading %s"%txtfile, end="")
    File = Client.files.create(
        file=open(datafolder+txtfile, "rb"),
        purpose='assistants'
)    
    print(" --- Uploaded")
    time.sleep(1)
    NewUploads.append(txtfile)
    NewUploadIDs.append(File.id)
with open(UploadListFile, 'a') as f:
    for item in NewUploads:
        f.write("%s\n" % item)
if len(NewUploads)>0:
    print("Updated %s (Names of the Uploaded Files)"%UploadListFile)
else:
    print("No new files uploaded")
# Append the new IDs to the file UploadIDFile
with open(UploadIDFile, 'a') as f:
    for item in NewUploadIDs:
        f.write("%s\n" % item)
if len(NewUploadIDs)>0:
    print("Updated %s (OpenAI IDs of the uploaded files)"%UploadIDFile)
else:
    print("No new IDs uploaded")
#
UploadID=UploadID+NewUploadIDs
UploadList=UploadList+NewUploads

Skipping the-requiem-for-a-dream-israels-untaken.txt because it has already been uploaded
Skipping the-importance-of-elite-consensus.txt because it has already been uploaded
Skipping openai-astounds-us-again.txt because it has already been uploaded
Skipping why-did-elon-musk-buy-twitter.txt because it has already been uploaded
Skipping elon-musk-the-spaceman.txt because it has already been uploaded
Skipping let-us-talk-about-elon-musk.txt because it has already been uploaded
Skipping despicable-acts-part-2.txt because it has already been uploaded
Skipping despicable-deeds.txt because it has already been uploaded
Skipping rogue-age-and-climate-change-unpredictable.txt because it has already been uploaded
Skipping rogue-age-accessory-1-population.txt because it has already been uploaded
Skipping rogue-renaissance-on-globe-with-upheavals.txt because it has already been uploaded
Skipping the-great-stagnation-ends-but-for.txt because it has already been uploaded
Skipping when-the-rivers-run

## Read Assistant ID from file or create new ##
Delete `data/general/data.txt` if you want to create a new assistant.  This code does not check if an assistant with that name already exists assigned to your account in OpenAI.  You can see the assistants assigned to you in [Assistants Page](https://platform.openai.com/assistants) after you log in to OpenAI.

In [6]:
# Check if the DataFile exists
if os.path.isfile(DataFile):
    # Read the Assistant ID from the file
    f = open(DataFile, "r")
    AssistantID = f.read()
    f.close()
    Assistant = Client.beta.assistants.retrieve(AssistantID)
    print("Retrieved Assistant with ID from OpenAI API: %s"%AssistantID)
else:
    # Create a new Assistant
    Assistant = Client.beta.assistants.create(
    name="Assistant for "+webfolder,
    model="gpt-4-1106-preview",
    instructions="You are a friendly AI chatbot. You answer queries using the information in the text files attached to the query message.  If the \
 response is not in the text files, you can respond with 'I don't know'.  Try to be informative with your answers.",
    tools=[{"type": "retrieval"}],
    # file_ids=UploadID
    )
    AssistantID = Assistant.id
    # Write the Assistant ID to the file
    f = open(DataFile, "w")
    f.write(AssistantID)
    f.close()
    print("Created Assistant ID: %s"%AssistantID)

Retrieved Assistant with ID from OpenAI API: asst_YfdA8UsATfJCHBZICrMV84aF


## Generate Embeddings ##
The OpenAI does not allow to attach more than 20 files.  I have more than 20 
blog posts that I want to be able to query.  I do not want to combine them together.  Since I cannot attach more than 20 to an Assistant, I will have to find the files that are most relevant to the question and attach only those files.  These files can be selected in a number of ways. Before we consider different options, we need to generate embeddings.  In the following cell, I do the following:
* Load embeddings from the `npz` file if it exists
* If there is no `npz` file, embed all files and store with metadata
* Check if there are new text files in the TextList
* Embed the new text files and append them to the end of the embeddings

In [7]:
def embedandsave():
    embeddings=embedpagelist(TextList)
    metadata={"model":LLM, "webfolder":webfolder, "textlist":TextListFile, "texts":TextList}
    np.savez(EmbeddingFile,data=embeddings, metadata=metadata)
    return embeddings, metadata

if not os.path.exists(EmbeddingFile):
    (embeddings, metadata)=embedandsave()
    print("Created %s"%EmbeddingFile)
# Load embeddings from file
with np.load(EmbeddingFile, allow_pickle=True) as data:
    embeddings=data['data']
    metadata=data['metadata'].item()

if not metadata["model"]==LLM:
    (embeddings, metadata)=embedandsave()
    print("Re-created %s because of the LLM change"%EmbeddingFile)   

# Generate the liost of embedded textx that are in the npz file:
embedded_texts=[]
for pair in metadata["texts"]:
    embedded_texts.append(pair[1])

# Now check if there are any new texts to be embedded
update=False
for (itext,pair) in enumerate(TextList):
    url=pair[0]
    texttitle=pair[1]
    # print("Check '"+texttitle+"' --> ", end="")
    if texttitle not in embedded_texts:
        print("Update "+texttitle)
        text=rdweb(url, None)
        sa=sepstr(text, SEP)
        for (isegment, s) in enumerate(sa):
            v=get_embedding(s)
            w=np.append(v,[itext,isegment])
            embeddings=np.vstack((embeddings,w))
        metadata["texts"].append(pair)
        update=True
    # else:
        # print("OK")
if update:
    # Save the embeddings to a file
    np.savez(EmbeddingFile,data=embeddings, metadata=metadata)
    print("Updated %s"%EmbeddingFile)
else:
    print("No new texts to be embedded")



No new texts to be embedded


## Special Questions ##

In [8]:
OpMode=0
OpTask=["check", "answer", "summarise", "quit"][OpMode]

def checkmode():
    global OpMode
    OpMode=0
    print("Switched to check mode")
    return
def answermode():
    global OpMode
    OpMode=1
    print("Switched to answer mode")
    return
def summarisemode():
    global OpMode
    OpMode=2
    print("Switched to summarise mode")
    return
def quitmode():
    global OpMode
    OpMode=3
    print("Quit")
    return
def setoptask():
    global OpTask
    OpTask=["check", "answer", "summarise", "quit"][OpMode]
    return

Specials=["Check", "Answer", "Summarise", "Quit"]
SpecialFunctions=[checkmode, answermode, summarisemode, quitmode]
SpecialEmbeds=[get_embedding(s) for s in Specials]
OpMode=0
OpTask=["check", "answer", "summarise", "quit"][OpMode]

def checkmode():
    global OpMode
    OpMode=0
    print("Switched to check mode")
    return
def answermode():
    global OpMode
    OpMode=1
    print("Switched to answer mode")
    return
def summarisemode():
    global OpMode
    OpMode=2
    print("Switched to summarise mode")
    return
def quitmode():
    global OpMode
    OpMode=3
    print("Quit")
    return
def setoptask():
    global OpTask
    OpTask=["check", "answer", "summarise", "quit"][OpMode]
    return

Specials=["Check", "Answer", "Summarise", "Quit"]
SpecialFunctions=[checkmode, answermode, summarisemode, quitmode]
SpecialEmbeds=[get_embedding(s) for s in Specials]


## Prompt ##
In the thread, I will use the Assistant instructions that are already set.

## Logging our session ##
I will store the query, the associated files, and the OpenAI response in a file.  This file will be in markdown format.

In [9]:
Logging=True

import datetime
today = datetime.date.today()
#
if Logging:
    Logfile=datafolder+webfolder+"_log.md"
    Log=open(Logfile, "a")
    Log.write(today.strftime("%B %d, %Y")+"\n\n")
    Log.close()

In [10]:
def log(s):
    if Logging:
        Log=open(Logfile, "a")
        Log.write(s+"\n\n")
        Log.close()
    return

## Operation Mode ##
This can do one of the following things depending on the value of the `OpMode`.  

|Mode|Action|
|:--:|----|
|0 - Check|Do not answer. Only return the URLs of the pages that match |
|1 - Answer|Find the best-match URLs, ask OpenAI to reply using thuse URL files|
|2 - Summarise|Find the best-match URLs, ask OpenAI to summarise them| 

The default mode is `check`.  One can change the mode by entering one of the following strings (or similar) as the query:

In [11]:
table="|Query text|`OpTask`|\n|--|--|\n"
for s in Specials:
    table+="|%s|%s|\n"%(s, ["Check", "Answer", "Summarise", "Quit"][Specials.index(s)])
md(table)


# If the query text is similar to one of the special texts,
# then switch to the corresponding mode
# Returns TRUE if mode is switched; FALSE otherwise
def setmode(query_embedding):
    for (i,s) in enumerate(Specials):
        if cosine_similarity(query_embedding, SpecialEmbeds[i])>0.9:
            # print("setmode: i=%d, s=%s"%(i,s))
            SpecialFunctions[i]()
            setoptask()
            return True
    return False

|Query text|`OpTask`|
|--|--|
|Check|Check|
|Answer|Answer|
|Summarise|Summarise|
|Quit|Quit|


## Select which files to attach ##
The easiest thing to do is to attach the top N files that are the most relevant to the query. The top N files are determined by computing the cosine-similarity of their segments to the query.

The file score will be equal to the score of the closest segment in that file.

In [12]:

def findtopN(N, query_embedding):
    global TextSimilarity, topN
    TextSimilarity=np.ones(len(embeddings))*(-1)
    for i in range(len(embeddings)):
        similarity = cosine_similarity(embeddings[i][:-2], query_embedding)
        itext=int(embeddings[i][-2])
        if similarity>TextSimilarity[itext]:
            TextSimilarity[itext]=similarity
    topN=(np.argsort(TextSimilarity)[::-1])[0:N]
    topN=topN.astype(int)
    topN=topN.tolist()


## Show the top N matching URLs ##

In [13]:
def prtopN():
    s="**Query** : "+Query
    s=s+"\n\nThe following are the top %d most relevant texts:\n\n"%N
    s=s+"|itext|similarity|title|\n|--|--|--|\n"
    for i in topN:
        s=s+"|%d|%f|%s|\n"%(i,TextSimilarity[i],metadata["texts"][i][1])
    s=s+"\n\nWe will attach these files to the thread.\n\n"
    md(s)
    log(s)

def showcheckpages(query):
    s="**TOPIC** : "+query+"\n\n"
    for i in range(0,3):
        s=s+"**PAGE** : %s\n\n"%(metadata["texts"][topN[i]][1])
        s=s+"**URL** : %s\n\n"%(metadata["texts"][topN[i]][0])
    md(s)
    log(s)

def showanswer(query):
    msg=Client.beta.threads.messages.list(thread_id=Thread.id)
    s="\n\n**Query** : "+query+"\n\n"
    s=s+"**ANSWER** : "+msg.data[0].content[0].text.value
    md(s)
    log(s)

## Create Thread ##
Create a thread with the query and the top N relevant files as information. In `qa.ipynb`, I attached files to the thread and I will continue this way even though in `playground.ipynb`, the files were attached to the Assistant.

If thread attachment does not work, then I will attach files to the Assistant.  This is done by the [Modify Assistant](https://platform.openai.com/docs/api-reference/assistants/modifyAssistant) call. A list of File IDs can be attached this way. It sounds like this new file list overwrites the previous list. The API notes say that
>If a file was previously attached to the list but does not show up in the list, it will be deleted from the assistant.

In [14]:
def threadfiles():
    return [UploadID[i] for i in topN]

# First question in this topic
def startthread(query):
    global Thread, RunIDs
    Thread = Client.beta.threads.create(messages=[
          {
            "role": "user",
            "content": query,
            "file_ids": threadfiles()
          }
        ]) 
    RunIDs=[] # When the thread is reset, the Run #ID list is also reset
    return
  
# Subsequent questions in the same topic
def qtothread(query):
    global Thread

    thread_message = Client.beta.threads.messages.create(
      Thread.id,
      role="user",
      content=query,
      file_ids=threadfiles()
    )
     
    



## Run the Query ##

In [15]:
def checkfiles(run):
    # Check the files attached to the run against the files in the thread
    # return True if the files are the same; False otherwise
    # print("Checking files")
    nrunfiles=len(run.file_ids)
    threadfiles= [UploadID[i] for i in topN]
    nthreadfiles=len(threadfiles)
    if nrunfiles!=nthreadfiles:
        return False
    else:
        for i in range(nrunfiles):
            if run.file_ids[i]!=threadfiles[i]:
                return False
        return True
    
def runquery():
    extrainstructionaltext="You have access to the following files:\n\n"+\
            "\nfile_ids: %s"%(threadfiles())+ "\n\nUse the files to answer the question."
    run = Client.beta.threads.runs.create(
            thread_id=Thread.id,
            assistant_id=AssistantID,
            instructions=Assistant.instructions)

    print("THREAD # : %s, RUN # : %s\nInstructions : %s"%(Thread.id, run.id, run.instructions), end="")
    RunIDs.append(run.id)

    print(run.status, end=" ")
    while run.status != "completed":
        run = Client.beta.threads.runs.retrieve(thread_id=run.thread_id, run_id=run.id)
        print(run.status[0], end="")
        if run.status == "completed":
            print(" \u2713", checkfiles(run))
            break
        else:
            time.sleep(1)

    return run


    
# print(messages.data[0].content[0].text.value)

In [16]:
# This function lists the responses for all the runs on the current thread
def show_all_messages():
    s=""
    for i, run_id in enumerate(RunIDs):
        steps_page=Client.beta.threads.runs.steps.list(run_id=run_id, thread_id=Thread.id)
        message_id=steps_page.data[0].step_details.message_creation.message_id
        message=Client.beta.threads.messages.retrieve(thread_id=Thread.id, message_id=message_id)
        s=s+"%02d - %s - %s\n\n%s\n\n"%(i,Thread.id, run_id, message.content[0].text.value)
        annotations=message.content[0].text.annotations
        s+="\n\n%s\n\n"%annotations
    md(s)

In [17]:
def chatloop():
    global Query, Run, Thread
    while(OpTask!="quit"):
        s=input("Enter command: ")
        sembed=get_embedding(s)
    # Check if the Query is similar to one of the strings in Specials:
        if setmode(sembed):
            continue # Continue to get the next command
        if s!="": # If empty string, use the last Query; otherwise update the Query
            Query=s
            query_embedding=get_embedding(Query)
        if OpTask=="check":
            findtopN(N, query_embedding)
            prtopN()
            showcheckpages(Query)
            continue
        if OpTask=="answer":
            if Thread==None:
                findtopN(N, query_embedding)
                prtopN()
                startthread(Query)
            else:
                qtothread(Query)
            Run=runquery()
            showanswer(Query)
            
chatloop()
    

Quit


In [18]:

thread_messages = Client.beta.threads.messages.list(Thread.id)
message_list=list(thread_messages)
# print(thread_messages.data)
# pr_objfields(thread_messages)

def tabulate_message(i):
    message=message_list[i]
    s="\n\n|Attribute|Value|\n|---|---|\n"
    s+="|ID|`%s`|\n"%message.id
    s+="|Created|`%s`|\n"%(ts2lt(message.created_at))
    s+="|Object name|`%s`|\n"%message.object
    s+="|assistant_id|`%s`|\n"%message.assistant_id
    s+="|thread_id|`%s`|\n"%message.thread_id
    s+="|run_id|`%s`|\n"%message.run_id
    s+="|file_ids|`%s`|\n"%message.file_ids
    s+="|role|`%s`|\n"%message.role
    s+="|content|`%s`|\n"%message.content
    mdc(s)
    mdc("\n\n")

mdc("\n\nThe following shows the fields of the six messages belonging to this thread. The messages are \
ordered in reverse. The last message \
is the initial question.  The second last message is the OpenAI answer to this question.")

for i in range(len(message_list)):
    mdc("\n\n#### Message %d ####\n\n"%i)
    tabulate_message(i)

AttributeError: 'NoneType' object has no attribute 'id'

In [ ]:
updated_thread = Client.beta.threads.update(
  Thread.id,
  extra_query={"query": "Who did that study?"},

  metadata={
    "modified": "true",
    "user": "Q2"
  }
)
print(updated_thread)


Thread(id='thread_Ql8V88Q4i0LXpXRngacVAxWg', created_at=1706683319, metadata={'modified': 'true', 'user': 'Q2'}, object='thread')
